In [29]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import VGG16
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [1]:
import numpy as np
from PIL import Image
import os
import h5py

### Building the model

In [3]:
image_model = VGG16(include_top=True, weights='imagenet')

VGG_last_layer = image_model.get_layer('fc2')
vgg_model = Model(inputs = image_model.input, outputs = VGG_last_layer.output)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Function to extract features from input image

In [21]:
def get_feature_vector(path, size=(224,224,)):
    img = Image.open(path)
    img = img.resize(size=size,resample=Image.LANCZOS)
    img = np.array(img)/255.0
    if len(img.shape)==2:
        img = np.repeat(img[:,:,np.newaxis], 3, axis=2)
    image_batch = np.expand_dims(img, axis=0)
    activations = vgg_model.predict(image_batch).reshape((4096,))
    return activations

### Building h5 file to store feature vectors of all images in database

In [5]:
batch = []
files = od.listdir('./train2017')
files.sort()
for file in files:
    activations = get_feature_vector('./train2017/'+file)
    batch.append(activations)
activation_file = h5py.File("vgg_activations.h5","w")
activation_file.create_dataset('last_layer_activations',data=vgg_activations)
activation_file.close()    

### Loading the generated h5 file

In [35]:
activation_file = h5py.File("vgg_activations.h5","r")
fvec = list(activation_file['last_layer_activations'])
fvec.append(features)
print("h5 file loaded")

h5 file loaded


### Taking user input to path

In [ ]:
user_image = input("Enter the image path:")
features = get_feature_vector(user_image)
img = mpimg.imread(user_image)
imgplot = plt.imshow(img)
plt.show()
print("Image feature vector loaded")

### Applying clustering algorithm

In [ ]:
files = os.listdir('./train2017')
files.sort()
l=[]
for i in range(0,118288,22578):
    nn = NearestNeighbors(n_neighbors=1,algorithm='brute')
    end = i+22578
    if end>118287:
        end = 118287
    nn.fit(fvec[i:end])
    neigh = nn.kneighbors(X=np.array([features]),return_distance=False)[0][0]
    l.append(files[i+neigh])
    print(l[-1])

### Displaying the model

In [ ]:
for img in l:
    img = mpimg.imread('./train2017/'+img)
    imgplot = plt.imshow(img)
    plt.show()